In [1]:
import torch
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)
print(device)

cuda


In [111]:
#Build dataset

with open("names.txt", 'r', encoding="UTF-8") as f:
    text = f.read()
    words = text.splitlines()
    chars_used = sorted(set("".join(words)))

stoi = {char:integer+1 for integer,char in enumerate(chars_used)}
itos = {integer+1:char for integer,char in enumerate(chars_used)}

stoi["."] = 0
itos[0] = "."

inputs, labels = [], []
for w in words[:]:
    chs = ["."] + list(w) + ["."]
    for char1, char2 in zip(chs, chs[1:]):
        ix1 = stoi[char1]
        ix2 = stoi[char2]

        inputs.append(ix1)
        labels.append(ix2)

inputs = torch.tensor(inputs)
labels = torch.tensor(labels)
W = torch.randn((len(stoi), len(stoi)), requires_grad=True)

In [134]:
epochs = 100
print(f"Training on {inputs.nelement()} examples")
one_hot_inputs = F.one_hot(inputs, num_classes=len(stoi)).type(torch.float32)
logits = one_hot_inputs @ W
counts = torch.exp(logits)
probs = torch.div(counts, counts.sum(dim=1, keepdim=True))
loss = -probs[torch.arange(inputs.nelement()), labels].log().mean()
print(f"Loss before --> {loss}")


for epoch in range(epochs):
    #Forward pass
    #One hot vectors are going through the neural network's neurons
    logits = one_hot_inputs @ W
    counts = torch.exp(logits)
    probs = torch.div(counts, counts.sum(dim=1, keepdim=True))
    loss = -probs[torch.arange(inputs.nelement()), labels].log().mean()

    #Zero the grad
    W.grad = None

    #Loss backward
    loss.backward()

    #//Optmizer.step
    #the number is basically the learning rate
    W.data += -1*W.grad

logits = one_hot_inputs @ W
counts = torch.exp(logits)
probs = torch.div(counts, counts.sum(dim=1, keepdim=True))
loss = -probs[torch.arange(inputs.nelement()), labels].log().mean()
print(f"Loss after epochs --> {loss}")





Training on 228146 examples
Loss before --> 2.454224109649658
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 0.]], device='cuda:0')
Loss after epochs --> 2.4542222023010254


In [201]:
#Sampling from the model

names = []
n_names = 10
ix = 0

for n in range(n_names):
    current_name = ""
    while True:
        logits = one_hot_inputs @ W
        counts = torch.exp(logits)
        probs = torch.div(counts, counts.sum(dim=1, keepdim=True))
        p = probs[ix, :]
        ix = torch.multinomial(p, num_samples=1, replacement=True).item()
        if ix == 0:
            names.append(current_name)
            break
        else:
            current_name += itos[ix]

print(names)





['eadvozeaieziaveedrtaed', 'mstolaniar', 'wi', 'shs', 'fhar', 'slwabbaiohniyaers', 'niiar', 'traeomr', 'knaekcehlsieeedr', 'niiear']
